# StrangleBot
We are building a trading bot to execute short strangles and long strangles on S&P 500 companies based on features we can extract from yfinance data.

## Setup

`git clone https://github.com/PeterFavero/cqf_final_project.git`

`cd cqf_final_project`

`python3 -m venv venv`

MacOS/Linux: `source venv/bin/activate` or Windows: `venv\Scripts\activate`

`pip3 install -r requirements.txt`

In [56]:
# IMPORTS
# ---------------------------------------------
import yfinance as yf
import pandas as pd
import pytz
import ssl
from datetime import datetime, timedelta
from tqdm import tqdm
import sys
import os
import numpy as np

In [57]:
# ENVIROMENT CONFIGURATION
# ---------------------------------------------
# This is nescessary for web scraping
ssl._create_default_https_context = ssl._create_unverified_context

In [58]:
# CONSTANTS
# ---------------------------------------------
SP500_TICKER = "^GSPC"
TEN_YEARS_AGO = pd.Timestamp(datetime.now() - timedelta(days=365*10)).replace(tzinfo=pytz.UTC)
ORIGINAL_STDOUT = sys.stdout
SUPPRESSED_STDOUT = open(os.devnull, 'w')

In [59]:
# DATA: HELPER FUNCTIONS 
# ---------------------------------------------

def parse_nasdaqlist():
    output_array = []  # Array to hold the extracted parts of lines
    with open('data/nasdaqlisted.txt', 'r') as file:  # Open the file in read mode
        next(file)  # Skip the first line
        for line in file:  # Iterate through each line in the file after the first
            # Find the position of the first '|' character
            separator_index = line.find('|')
            if separator_index != -1:  # If '|' is found
                # Extract the part of the line before the '|' character
                substring = line[:separator_index].strip()
                output_array.append(substring)  # Append to the output array
            else:
                # Handle cases where there is no '|'
                output_array.append(line.strip())
    return output_array

def ticker_listing_age(ticker, hist_dict) : 
    hist = hist_dict[ticker]
    return hist.index[0]

def data_of_tickers(full_ticker_strings_list) :
    cleaned_ticker_strings_list =  [ticker for ticker in full_ticker_strings_list if "." not in ticker]
    cleaned_ticker_objects_dict = {}
    cleaned_ticker_hists_dict = {}
    delisted_ticker_strings_list = []
    for ticker in tqdm(cleaned_ticker_strings_list, desc="Downloading price history data") :
        try :
            cleaned_ticker_objects_dict[ticker] = yf.Ticker(ticker)
            cleaned_ticker_hists_dict[ticker] = cleaned_ticker_objects_dict[ticker].history(period='2y')
        except Exception as _ :
            delisted_ticker_strings_list.appened(ticker)
    return cleaned_ticker_strings_list, cleaned_ticker_objects_dict, cleaned_ticker_hists_dict, delisted_ticker_strings_list


In [60]:
# DATA: GET TICKERS LIST
# ---------------------------------------------

sp500_unprocessed = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_list()
nasdaq_unprocessed = parse_nasdaqlist()
sp500_and_nasdaq_unprocessed = list( set(sp500_unprocessed) | set(nasdaq_unprocessed) )

In [61]:
# DATA: DATA LOADING
# ---------------------------------------------

ticker_strings_list, ticker_objects_dict, ticker_hists_dict, delisted_ticker_strings_list = data_of_tickers(sp500_and_nasdaq_unprocessed)

In [ ]:
# DATA ANALYSIS: HELPER FUNCTIONS
# ---------------------------------------------

def calculate_single_historical_volatility(ticker, start_date, end_date):
    """
    Calculate the historical volatility for a given stock ticker over the entire specified period.
    Parameters:
    ticker (str): Stock ticker symbol.
    start_date (str): Start date for data retrieval (format: 'YYYY-MM-DD').
    end_date (str): End date for data retrieval (format: 'YYYY-MM-DD').
    Returns:
    float: Historical volatility over the specified period.
    """
    # Fetch historical stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    # Calculate daily returns
    data['returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
    # Calculate the standard deviation of returns
    volatility = data['returns'].std() * np.sqrt(252)  # Annualize the volatility
    return volatility